<a href="https://colab.research.google.com/github/dernameistegal/airbnb_price/blob/main/just_a_scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path= ("/content/drive/MyDrive/Colab/airbnb/data/translations/translated_listings.pickle")
listings = pd.read_pickle(path)

In [37]:
# transform price
listings["price"] = (
    listings["price"].str.replace("$", "").str.replace(",", "").astype(float)
)

In [4]:
import torch 
from transformers import BertTokenizer, BertModel


class Language_Model(torch.nn.Module):
    def __init__(self, freeze=True):
        super().__init__()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained("bert-base-uncased")
        for param in self.model.parameters():
            param.requires_grad = False

    def forward(self, text):
        encoded_input = self.tokenizer(text, return_tensors='pt')
        output = self.model(**encoded_input)
        return output

In [31]:
from torch.utils.data import DataLoader, Dataset

class Airdataset(Dataset):
    def __init__(self, data, column):
        super().__init__()
        self.data = data
        self.column = column

    def __getitem__(self, key):
        return int(self.data[self.column].iloc[key]), self.data.index[key]

    def __len__(self):
        return len(self.data)

In [53]:
class mean_Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mean = torch.nn.Parameter(torch.ones(1))

    def forward(self, x):
        ret = x*0 + self.mean
        return ret

In [54]:
data = Airdataset(listings, ["price"])
train_size, test_size = int(0.1 * len(data)), int(0.1 * len(data))
rest = len(data) - train_size - test_size
traindata, valdata, _ = torch.utils.data.random_split(data, [train_size, test_size, rest])
trainloader =  DataLoader(traindata, batch_size = 1000)
valloader =  DataLoader(valdata, batch_size = 1000)

In [76]:
def train(epochs, trainloader, valloader, model, loss_fn, optimizer, device, scheduler):
    for epoch in range(epochs):
        model.train()
        for price, id in trainloader:
              price = price.float().to(device)
              optimizer.zero_grad()
              pred = model(price)
              loss = loss_fn(pred, price)
              loss.backward()
              optimizer.step()
        scheduler.step()
        with torch.no_grad():
            model.eval()
            loss = []
            for price, id in valloader:
                pred = model(price)
                loss.append(loss_fn(pred, price))

            print(np.sum(loss)/len(traindata))

In [ ]:
def stupid_model_prediction(y_train, y_test):
    a = np.empty(len(test_y))
    a.fill(np.mean(train_y))
    return a


pred_y = stupid_model_prediction(traindata, testdata)

In [77]:
#model = mean_Model()
optimizer = torch.optim.Adam(model.parameters(), lr = 5)
loss_fn = torch.nn.MSELoss()
device = "cpu"
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 30)
train(100, trainloader, valloader, model, loss_fn, optimizer, "cpu", scheduler)

112.19677905701755
112.14838952850877
112.10602384868422
112.0943393640351
112.10331688596492
112.12410910087719
112.14490131578947
112.15375548245613


KeyboardInterrupt: ignored

In [64]:
model.mean

Parameter containing:
tensor([2.9964], requires_grad=True)